# Flight Data Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [56]:
import influxdb_client_3 as InfluxDBClient3
import json
import time
from IPython.display import display, HTML
import polars as pl
from dash import Dash, html, dcc
import plotly.express as px

The class `query_timer` provides a simple stop watch function for query performance testing. This will be utilised later within the notebook.

In [53]:
class query_timer:
    def __init__(self):
        self.start_time = time.time()
        self.end_time = None
        self.elapsed_time = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        self.elapsed_time = self.end_time - self.start_time
        print('Execution time:', self.elapsed_time, 'seconds')
        return self.elapsed_time


Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [54]:
token = ''
host = ''    
org = 'Jay-IOx'
db = 'flightdemo'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    


In [55]:
query = "SELECT * FROM \"flight\" WHERE time >= now() -  interval '8 days' limit 10000" 
table = client.query(query=query, language="sql")
print(table)

pyarrow.Table
baro_altitude: double
callsign: string
category: int64
geo_altitude: double
icao24: string
last_contact: int64
latitude: double
longitude: double
on_ground: bool
origin_country: string
position_source: int64
sensors: string
spi: bool
squawk: string
time: timestamp[ns] not null
time_position: int64
true_track: double
velocity: double
vertical_rate: double
----
baro_altitude: [[4305.3,4305.3,4305.3,4305.3,4305.3,...,10972.8,10972.8,10972.8,11887.2,11887.2],[6606.54,6606.54,6606.54,6606.54,6606.54,...,4671.06,10972.8,10972.8,10972.8,10972.8]]
callsign: [["","","","","",...,"SWG686  ","SWG686  ","SWG686  ","WJA140  ","WJA140  "],["","","","","",...,"AZU4719 ","AZU4508 ","AZU4508 ","AZU4508 ","AZU4508 "]]
category: [[0,0,0,0,0,...,1,1,1,1,1],[0,0,0,0,0,...,3,4,4,4,4]]
geo_altitude: [[233,233,233,233,233,...,11109.96,11109.96,11109.96,12108.18,12108.18],[3255,3255,3255,3255,3255,...,4922.52,11574.78,11574.78,11574.78,11574.78]]
icao24: [["d01073","d01073","d01073","d01073","d01

The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

Return all columns from the table flight for the past 3 days.

In [57]:
df = pl.from_arrow(table)
display(df)

baro_altitude,callsign,category,geo_altitude,icao24,last_contact,latitude,longitude,on_ground,origin_country,position_source,sensors,spi,squawk,time,time_position,true_track,velocity,vertical_rate
f64,str,i64,f64,str,i64,f64,f64,bool,str,i64,str,bool,str,datetime[ns],i64,f64,f64,f64
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:42:43.528557,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:43:09.460129,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:43:35.215190,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:44:00.959938,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:44:26.678951,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:44:52.456026,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:45:21.733319,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:45:49.460784,1684762955,0.0,0.0,0.0
4305.3,"""""",0,233.0,"""d01073""",1684762955,51.4734,6.4363,false,"""<NA>""",3,"""None""",false,"""0142""",2023-05-22 14:46:16.234436,1684762955,0.0,0.0,0.0


In [58]:
df = df.groupby("origin_country").agg(
        [
            pl.count("icao24"),
            pl.max("velocity")
        ]
    )
display(df)

origin_country,icao24,velocity
str,u32,f64
"""Belarus""",34,223.53
"""Bangladesh""",73,259.4
"""<NA>""",204,146.24
"""Austria""",2689,264.2
"""Bulgaria""",186,233.57
"""Angola""",12,239.17
"""Bahrain""",122,250.5
"""Albania""",18,229.17
"""Argentina""",284,245.55


In [59]:
# then let's load some csv data with information about pokemon
df = pl.read_csv(
    "https://gist.githubusercontent.com/ritchie46/cac6b337ea52281aa23c049250a4ff03/raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv"
)
display(df.head())

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false
3,"""VenusaurMega V…","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false


In [60]:
out = df.select(
    [
        "Type 1",
        "Type 2",
        pl.col("Attack").mean().over("Type 1").alias("avg_attack_by_type"),
        pl.col("Defense")
        .mean()
        .over(["Type 1", "Type 2"])
        .alias("avg_defense_by_type_combination"),
        pl.col("Attack").mean().alias("avg_attack"),
    ]
)
display(out)

Type 1,Type 2,avg_attack_by_type,avg_defense_by_type_combination,avg_attack
str,str,f64,f64,f64
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Fire""",null,88.642857,58.3,75.349693
"""Fire""",null,88.642857,58.3,75.349693
"""Fire""","""Flying""",88.642857,82.0,75.349693
"""Fire""","""Dragon""",88.642857,111.0,75.349693
"""Fire""","""Flying""",88.642857,82.0,75.349693


In [61]:
filtered = df.filter(pl.col("Type 2") == "Psychic").select(
    [
        "Name",
        "Type 1",
        "Speed",
    ]
)
display(filtered)

Name,Type 1,Speed
str,str,i64
"""Slowpoke""","""Water""",15
"""Slowbro""","""Water""",30
"""SlowbroMega Sl…","""Water""",30
"""Exeggcute""","""Grass""",40
"""Exeggutor""","""Grass""",55
"""Starmie""","""Water""",115
"""Jynx""","""Ice""",95


In [62]:
out = filtered.with_columns(
    [
        pl.col(["Name", "Speed"]).sort_by("Speed", descending=True).over("Type 1"),
    ]
)
display(out)

Name,Type 1,Speed
str,str,i64
"""Starmie""","""Water""",115
"""Slowbro""","""Water""",30
"""SlowbroMega Sl…","""Water""",30
"""Exeggutor""","""Grass""",55
"""Exeggcute""","""Grass""",40
"""Slowpoke""","""Water""",15
"""Jynx""","""Ice""",95


In [63]:
fig2 = px.bar(out, x=0, y=2, color=1, title='test')

display(fig2)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': '1=Water<br>0=%{x}<br>2=%{y}<extra></extra>',
              'legendgroup': 'Water',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Water',
              'offsetgroup': 'Water',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Starmie', 'Slowbro', 'SlowbroMega Slowbro', 'Slowpoke'], dtype=object),
              'xaxis': 'x',
              'y': array([115, 30, 30, 15], dtype=object),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': '1=Grass<br>0=%{x}<br>2=%{y}<extra></extra>',
              'legendgroup': 'Grass',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Grass',
              'offsetgroup': 'Grass',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Exeggutor', 'Exeggcute'], dtype=object),
              'xaxis': 'x',
              'y': array([55, 40], dtype=object),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': '1=Ice<br>0=%{x}<br>2=%{y}<extra></extra>',
              'legendgroup': 'Ice',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': 'Ice',
              'offsetgroup': 'Ice',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Jynx'], dtype=object),
              'xaxis': 'x',
              'y': array([95], dtype=object),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': '1'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'test'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': '0'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': '2'}}}
})